# RUN: CoV Postive Controls

```
Lead     : ababaian
Issue    : na
Version  : 5f03d2fe21ae31e285e7c88854514af1d9bf5d71
start    : 2020 04 27
complete : 2020 04 28
files    : ~/serratus/notebook/200427_ab/
s3_files : s3://serratus-public/notebook/200427_ab/
output   : s3://serratus-public/out/200427_cov_positive/
```

### Objectives
- There are ~3000 'annotated' coronavirus SRA accession on NCBI, this is a first pass analysis at the 862 illumina/bgi accessions
- Create a "base" set of aligned bam files for the assembly team to work with.


## Serratus Initialization

Local system initialization procedures for `serratus`

In [1]:
date

Mon Apr 27 18:09:36 PDT 2020


### SRA Search
On: `https://www.ncbi.nlm.nih.gov/sra`

These are the search terms used as SRA query and respective matches:

`txid694002[Organism:exp]` : 3086

*Platform Breakdown:
```
  BGISEQ(22)
  Capillary(1,787)
  Illumina(840)
  Ion Torrent(26)
  Oxford Nanopore(411)
```

`"platform illumina"[Properties] OR "platform bgiseq"[Properties]  AND txid694002[Organism:exp]` : 862


`"platform illumina"[Properties] AND txid694002[Organism:exp]` : 840

`"platform illumina"[Properties] AND "cloud s3"[Properties] AND txid694002[Organism:exp]` : 827

`"type_rnaseq"[Filter] "platform illumina"[Properties] AND "cloud s3"[Properties] AND txid694002[Organism:exp]` : 56

Note: many/most libraries are not annotated as RNA-seq but instead "whole genome sequencing" since they consider RNA virus genome whole genome sequencing.

#### Final Selection
862 Libraries. Send to --> File --> RunInfo

`"platform illumina"[Properties] OR "platform bgiseq"[Properties]  AND txid694002[Organism:exp]`

(saved temporarily in  ~/serratus/notebook/200427_ab/)

### Initialize local workspace


In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS
git rev-parse HEAD # commit version

d8943afd6735229cefb30efc8a7fa844fe1b6ae0


In [2]:
# Create local run directory
WORK="$SERRATUS/notebook/200427_ab"
mkdir -p $WORK; cd $WORK

In [3]:
# SRA RunInfo Table for run
RUNINFO="$WORK/CoV_positive_SraRunInfo.csv"
cat $RUNINFO

Run,ReleaseDate,LoadDate,spots,bases,spots_with_mates,avgLength,size_MB,AssemblyName,download_path,Experiment,LibraryName,LibraryStrategy,LibrarySelection,LibrarySource,LibraryLayout,InsertSize,InsertDev,Platform,Model,SRAStudy,BioProject,Study_Pubmed_id,ProjectID,Sample,BioSample,SampleType,TaxID,ScientificName,SampleName,g1k_pop_code,source,g1k_analysis_group,Subject_ID,Sex,Disease,Tumor,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash
SRR11514750,2020-04-14 12:10:08,2020-04-10 13:48:50,947580,161748802,947580,170,54,,https://sra-download.ncbi.nlm.nih.gov/traces/sra61/SRR/011244/SRR11514750,SRX8086360,LK1,AMPLICON,RANDOM,VIRAL RNA,PAIRED,0,0,ILLUMINA,Illumina iSeq 100,SRP254488,PRJNA616147,,616147,SRS6395995,SAMN14483189,simple,2697049,Severe acute respiratory syndrome coronavirus 2,EPI_ISL_417917,,,,,,,no,,,,,UNIVERSITY MALAYA,SRA1064092,,public,538DC4F384FFB5026FB05F615FA0BFFD,2BDBDC6AEE1B9473C51B88C4FE0E

### Packer / AMI Initialization (optional)
Does not need to be ran each time if you have access to the AMI already.

Current Build: `us-east-1: ami-046baafb2ee438b69`

In [ ]:
cd $SERRATUS/packer
packer build docker-ami.json

### Build Serratus containers (optional)
Serratus containers are available on the `serratusbio` dockerhub. If you wish to deploy your own containers, you will have to build them from the `serratus` repository and upload them to your own dockerhub account.

This can be done with the `build.sh` script

In [ ]:
cd $SERRATUS

# If you want to upload containers to your repository
# include this.
export DOCKERHUB_USER='serratusbio' # optional
sudo docker login # optional

# Build all containers and upload them docker hub repo
# (if available)
./container_build.sh

### Terraform Initialization
The Global Variables for Terraform file must be modified to initialize for your system.

File: `$SERRATUS/terarform/main/terraform.tfvars`

This step must be done manually in a text editor currently.

In [ ]:
## Change these parameters in
## $SERRATUS/terarform/main/terraform.tfvars

# Your public IP followed by "/32"
# use: `curl ipecho.net/plain; echo`
LOCALIP="75.155.242.67/32" #dev_cidrs
# Your AWS key name
KEYNAME="serratus"         #key_name
# Dockerhub account containing serratus containers
DOCKERHUB_USER='serratusbio'    #dockerhub_account (optional)

In [ ]:
## CUSTOM CHANGES TO TERRAFORM
## $SERRATUS/terarform/main/main.tf
##
## main.tf
## Line 158:   instance_type      = "t3.medium"
## Line 159:   spot_price         = 0.02
## Line 167:   options            = "-k ${module.work_bucket.name} -b s3://serratus-public/out/200427_CoV_positve_ctrl"
##


In [18]:
# Initialize terraform
cd $SERRATUS/terraform/main
terraform init


Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corresponding provider blocks in configuration, with the constraint strings
suggested below.

* provider.local: version = "~> 1.4"

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [20]:
# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

module.download.data.aws_availability_zones.all: Refreshing state...
module.merge.data.aws_region.current: Refreshing state...
module.monitoring.aws_ecs_cluster.monitor: Refreshing state... [id=arn:aws:ecs:us-east-1:797308887321:cluster/serratus-monitor]
module.scheduler.data.aws_ami.amazon_linux_2: Refreshing state...
module.align.aws_cloudwatch_log_group.g: Refreshing state... [id=serratus-align]
module.merge.data.aws_availability_zones.all: Refreshing state...
module.download.aws_cloudwatch_log_group.g: Refreshing state... [id=serratus-dl]
module.work_bucket.aws_s3_bucket.work: Refreshing state... [id=tf-serratus-work-20200428162450548500000001]
module.monitoring.data.aws_ami.ecs: Refreshing state...
module.align.data.aws_region.current: Refreshing state...
module.scheduler.module.iam_role.aws_iam_role.role: Refreshing state... [id=SerratusIamRole-scheduler]
module.scheduler.data.aws_region.current: Refreshing state...
module.align.module.iam_role.aws_iam_role.role: Refreshing state

## Running Serratus 
Upload the run data, scale-out the cluster, monitor performance.


### Run Monitors & Upload table

Open SSH tunnels to monitor node then open monitors in browser

- [Scheduler Table](localhost:8000/jobs/)
- [Cluster Monitor: Grafana](http://localhost:3000/?orgId=1)
- [Cluster Monitor: Prometheus](http://localhost:9090)


#### Empty Scheduler Table (localhost:8000/jobs/)
![Empty Table Load Screen](200423_ab/empty_scheduler.png)

#### Ready Scheduler Table (localhost:8000/jobs/)
![Empty Table Load Screen](200423_ab/ready_scheduler.png)

In [21]:
cd $SERRATUS/terraform/main

# Open SSH tunnels to the monitor
./create_tunnels.sh

Tunnels created:

    localhost:3000 -- grafana

    localhost:9090 -- prometheus

    localhost:8000 -- scheduler


In [22]:
# Load SRA Run Info into scheduler (READY)
curl -s -X POST -T $RUNINFO localhost:8000/jobs/add_sra_run_info/

{
  "inserted_rows": 870, 
  "total_rows": 870
}


### Scale up the cluster
This will set-up 10 download, 10 align and 2 merge nodes to process data


In [26]:
./dl_set_capacity.sh 5

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name tf-asg-tf-serratus-dl-20200428162520347400000008 --desired-capacity 5


In [32]:
./align_set_capacity.sh 225

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name tf-asg-tf-serratus-align-20200428162520345900000007 --desired-capacity 225


In [50]:
./merge_set_capacity.sh 3

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name tf-asg-tf-serratus-merge-20200428162520357600000009 --desired-capacity 3


In [46]:
# Error fixes (manually help along)

curl -X POST "localhost:8000/jobs/split/601?state=split_done&N_paired=73&N_unpaired=0"

{
  "inserted_rows": 73, 
  "result": "success", 
  "total_rows": 7634
}


In [48]:
curl -X POST "localhost:8000/jobs/split/599?state=new&N_paired=0&N_unpaired=0"

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>400 Bad Request</title>
<h1>Bad Request</h1>
<p>The browser (or proxy) sent a request that this server could not understand.</p>


You can track the progress of accessions in the scheduler:

![Running Scheduler](200423_ab/running_scheduler2.png)

And monitor the performance of the cluster in the monitor:

![Running Monitor](200423_ab/running_monitor.png)

In [51]:
# When all downloading/splitting is done,
# scale-in the downloaders
./dl_set_capacity.sh 0

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name tf-asg-tf-serratus-dl-20200428162520347400000008 --desired-capacity 0


In [52]:
# When all alignment is done,
# scale-in the aligners
./align_set_capacity.sh 0

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name tf-asg-tf-serratus-align-20200428162520345900000007 --desired-capacity 0


In [53]:
# When all merging is done,
# scale in the mergers
./merge_set_capacity.sh 0

+ export AWS_REGION=us-east-1
+ AWS_REGION=us-east-1
+ aws autoscaling set-desired-capacity --auto-scaling-group-name tf-asg-tf-serratus-merge-20200428162520357600000009 --desired-capacity 0


In [54]:
# Dump the Scheduler SQLITE table to a local file
curl localhost:8000/db > \
  $SERRATUS/notebook/200427_ab/schedDump_run1.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2656k  100 2656k    0     0   937k      0  0:00:02  0:00:02 --:--:--  937k


## Shutting down procedures

Closing up shop.


### Save output of runs


In [ ]:
# Output files are in two folders:
# Bam and Bai files
#aws s3 ls s3://tf-serratus-work-<id>/out/bam/
# Flagstat and RefCount files
#aws s3 ls s3://tf-serratus-work-<id>/out/flagstat/


In [ ]:
# Copy output to a permenant bucket
# TODO: automatically transfer final outputs
# to the permenant bucket
#aws s3 sync \
#  s3://tf-serratus-work-<id>/out \
#  s3://serratus-public/out/YYMMDD_ii/


In [56]:
aws s3 cp \
  s3://tf-serratus-work-20200428162450548500000001/bam-blocks/SRR8265748/SRR8265748.9.bam \
  s3://serratus-public/tmp/

aws s3 cp \
  s3://tf-serratus-work-20200428162450548500000001/bam-blocks/SRR8265754/SRR8265754.9.bam \
  s3://serratus-public/tmp/

copy: s3://tf-serratus-work-20200428162450548500000001/bam-blocks/SRR8265748/SRR8265748.9.bam to s3://serratus-public/tmp/SRR8265748.9.bam
copy: s3://tf-serratus-work-20200428162450548500000001/bam-blocks/SRR8265754/SRR8265754.9.bam to s3://serratus-public/tmp/SRR8265754.9.bam


In [59]:
cd $SERRATUS

aws s3 ls s3://serratus-public/out/200427_CoV_positve_ctrl/summary/ > summary.list
wc -l summary.list

745 summary.list


## Destroy Cluster

Close out all resources with terraform (will take a few minutes).


In [61]:
cd $SERRATUS/terraform/main

terraform destroy -auto-approve
# WARNING this will also delete the standard output bucket/data
# Save data prior to destroy
# Can take a few minutes to complete.

module.scheduler.data.aws_region.current: Refreshing state...
module.align.module.iam_role.aws_iam_role.role: Refreshing state... [id=SerratusIamRole-serratus-align]
module.download.data.aws_ami.amazon_linux_2: Refreshing state...
module.download.module.iam_role.aws_iam_role.role: Refreshing state... [id=SerratusIamRole-serratus-dl]
module.scheduler.data.aws_ami.amazon_linux_2: Refreshing state...
module.scheduler.module.iam_role.aws_iam_role.role: Refreshing state... [id=SerratusIamRole-scheduler]
module.monitoring.aws_iam_role.task_role: Refreshing state... [id=SerratusIamRole-monitor]
module.monitoring.data.aws_ami.ecs: Refreshing state...
module.align.data.aws_region.current: Refreshing state...
module.monitoring.aws_ecs_cluster.monitor: Refreshing state... [id=arn:aws:ecs:us-east-1:797308887321:cluster/serratus-monitor]
module.align.data.aws_ami.amazon_linux_2: Refreshing state...
aws_security_group.internal: Refreshing state... [id=sg-03d3277136c12d1d6]
module.merge.data.aws_regi

In [ ]:
# Upload 

# Run Notes
Notes on this run for future reference.

## Errors

### Split Err 3 - Timeout (issue created)
` 	SRR11479046 	split_err 	2020-04-28 02:19:34.819629 	2020-04-28 02:23:08.818504 	i-059b34ac3dad98dba-2`

Gave error:
```

02:21:58
+ parallel --block 100M --pipe -N1000000 /home/serratus/s3_cp_formatted.sh s3://tf-serratus-work-20200428020557609100000001/fq-blocks/SRR11412326/SRR11412326.1.fq.%010d '{#}'

02:22:08
2020-04-28T02:22:08 fastq-dump.2.10.4 err: connection busy while creating file within network system module - error with https open 'https://sra-download.ncbi.nlm.nih.gov/traces/sra52/SRR/011210/SRR11479046'

02:23:08
2020-04-28T02:23:08 fastq-dump.2.10.4 err: timeout exhausted while creating file within network system module - error with https open 'https://sra-download.ncbi.nlm.nih.gov/traces/sra52/SRR/011210/SRR11479046'

02:23:08
2020-04-28T02:23:08 fastq-dump.2.10.4 err: timeout exhausted while creating file within network system module - failed to open 'SRR11479046'

02:23:08
fastq-dump (PID 2808) quit with error code 3
```
#### Also
```
22 	SRR11491770 	split_err 	2020-04-28 16:38:23.896767 	2020-04-28 16:41:08.466816 	i-036303ce58ac27c07-1 	N
23 	SRR11491769 	split_err 	2020-04-28 16:38:36.609181 	2020-04-28 16:41:08.319923 	i-036303ce58ac27c07-2
```

### Split Err  - Accession not found

- ERR3486809
- ERR3489114
- ERR3489845
- ERR3489846
- ERR3489847
- ERR3489104
- ERR3489105
- ERR3489106
- ERR3489112
- ERR3489113

```

02:21:36
2020-04-28T02:21:36 fastq-dump.2.10.4 err: name not found while resolving query within virtual file system module - failed to resolve accession 'ERR3486809' - no data ( 404 )

02:21:36
2020-04-28T02:21:36 fastq-dump.2.10.4 err: item not found while constructing within virtual database module - the path 'ERR3486809' cannot be opened as database or table

02:21:36
fastq-dump (PID 3715) quit with error code 3

02:21:36
++ error

02:21:36
++ echo Error encountered. Notifying the scheduler.

02:21:36
Error encountered. Notifying the scheduler.
```

Data on SRA:

```
Design: unspecified
Submitted by: Swedish University of Agricultural Sciences
Study: Swedish dogs with canine infectious respiratory disease highlight establishment of canine respiratory coronavirus in Sweden, and recent evolutionary history with bovine coronavirus
PRJEB34079 • ERP116929 • All experiments • All runs
show Abstract
Library:
Instrument: Illumina MiSeq
Strategy: Targeted-Capture
Source: METAGENOMIC
Selection: RANDOM
Layout: PAIRED
```

Perhaps the data is not public yet but it is listed in the SRA? Strange.

### Split hang..

```
SRR11059947 	splitting 	2020-04-28 18:13:32.988017 	None 	i-08afd13f2387ca687-2
```

The download and upload completed successfully, it looks like just that scheduler didn't update correctly.


All blocks are available on the work S3 bucket. It's purely a status issue.

```
...

18:27:01
+ echo '======= RUN COMPLETE ======='
...
```
Re-issue the CURL command: 

```
curl -s -X POST 'ec2-34-234-244-189.compute-1.amazonaws.com:8000/jobs/split/601?state=split_done&N_paired=73&N_unpaired=0'
```

### Split_err (Error 13) - Storage exhausted

Log:
```
599 	SRR11092063 	split_err 	2020-04-28 18:12:48.587515 	2020-04-28 18:57:05.029079 	i-01b620791bd42caf4-1
```
Error:
```

18:57:04
2020-04-28T18:57:04 fastq-dump.2.10.4 err: storage exhausted while writing file within file system module - system bad file descriptor error fd='8'

18:57:05
fastq-dump (PID 4498) was killed (signal 13)
fastq-dump (PID 4498) was killed (signal 13)
```

#### Also
```
822 	ERR233433 	split_err 	2020-04-28 18:49:38.875922 	2020-04-28 18:54:51.670510 	i-01b620791bd42caf4-2
```
```
832 	ERR330999 	split_err 	2020-04-28 18:54:51.684812 	2020-04-28 18:57:20.818245 	i-01b620791bd42caf4-2
```


### Merge Error
- Fixed in commit: 5f03d2f..e786da6

```
  Running -- run_merge.sh --
i-0fc2ae7b446f5ca4b-1 - Requesting job from Scheduler...
  i-0fc2ae7b446f5ca4b-1 - Wait State received.
Traceback (most recent call last):
  File "/home/serratus/serratus_summarizer.py", line 76, in <module>
    for Line in open(AccLenTaxFileName):
FileNotFoundError: [Errno 2] No such file or directory: './acc_len_taxid.txt'
samtools sort: failed to read header from "-"
samtools reheader: input file '-' must be BAM or CRAM
samtools index: "SRR11542243.bam" is in a format that cannot be usefully indexed
Failed to read header for "SRR11542243.bam"
  Post Merge Flagstat
  Uploading final bam data...
  Status: DONE
  i-0fc2ae7b446f5ca4b-2 - Job SRR11542243 is complete. Update scheduler.
 ```

### Merge Error - Disk Usage (~13x libraries premature EOF)
Noted by a spike in Disk Usage:

merge instances are very few, increase default to ~50 GB EBS so this error can always be avoided if a library is highly CoV+

Error tracking did not pick this up, there may be a handful of premature EOF files. When finished, sort files by size and the largest files are likely the ones containing thie error to be re-analyzed.

```
20:09:04
[E::bgzf_read] Read block operation failed with error 4 after 0 of 4 bytes

20:09:04
samtools index: failed to create index for "SRR2975521.bam": No such file or directory

20:09:08
[E::bgzf_read] Read block operation failed with error 4 after 0 of 4 bytes

20:09:08
[bam_flagstat_core] Truncated file? Continue anyway.

20:09:08
Post Merge Flagstat
```

### Merge Error - Put on S3 bucket

Upto 75 samples ran into error:

```
upload failed: ./SRR3742840.bam to s3://serratus-public/out/200427_CoV_positve_ctrl/bam/SRR3742840.bam An error occurred (AccessDenied) when calling the PutObject operation: Access Denied
```

I was messing around with bucket permissions of the `serratus-public` bucket and it triggered this erro without me noticing for ~30 minutes :'(

### Stuck on Align
```
516 	SRR8265748 	4234 	aligning 	2020-04-28 19:27:41.422791 	None 	i-027e2703a6dbc85c5-2
516 	SRR8265748 	4235 	aligning 	2020-04-28 19:27:53.245219 	None 	i-0d9cb16ecadb7448e-1
516 	SRR8265748 	4236 	aligning 	2020-04-28 19:27:53.854497 	None 	i-082f1abd5222890e3-2
516 	SRR8265748 	4237 	aligning 	2020-04-28 19:28:18.423503 	None 	i-0c7e40e1ca1052df4-1
516 	SRR8265748 	4238 	aligning 	2020-04-28 19:28:22.967331 	None 	i-087a06485c6ace4b4-1
516 	SRR8265748 	4239 	aligning 	2020-04-28 19:28:33.562454 	None 	i-087a06485c6ace4b4-2
516 	SRR8265748 	4240 	aligning 	2020-04-28 19:28:50.187048 	None 	i-0fd46d0e28e0620e6-2
516 	SRR8265748 	4241 	aligning 	2020-04-28 19:28:52.937474 	None 	i-0442b4a01a7f40a02-2
516 	SRR8265748 	4242 	aligning 	2020-04-28 19:28:53.531335 	None 	i-098a57ae03c2a4703-2
516 	SRR8265748 	4243 	done 	2020-04-28 19:28:55.775884 	2020-04-28 19:36:11.497497 	i-008869062ee3a9250-1
518 	SRR8265754 	4244 	aligning 	2020-04-28 19:29:04.017561 	None 	i-023046319f863c497-1
518 	SRR8265754 	4245 	aligning 	2020-04-28 19:29:12.762456 	None 	i-0540f07fccea13961-1
518 	SRR8265754 	4246 	aligning 	2020-04-28 19:29:17.873848 	None 	i-0d0cd2657eae790dc-1
518 	SRR8265754 	4247 	aligning 	2020-04-28 19:29:24.224857 	None 	i-01fdcffacbede1b7d-2
518 	SRR8265754 	4248 	aligning 	2020-04-28 19:29:27.046994 	None 	i-04e0b033251b2a811-1
518 	SRR8265754 	4249 	aligning 	2020-04-28 19:29:30.287140 	None 	i-0cc1f271026f37404-1
518 	SRR8265754 	4250 	aligning 	2020-04-28 19:29:38.214095 	None 	i-05fc1f6009d2aae8b-1
518 	SRR8265754 	4251 	aligning 	2020-04-28 19:29:41.901128 	None 	i-09d6ec6d0d63dea5e-2
518 	SRR8265754 	4252 	aligning 	2020-04-28 19:29:51.658913 	None 	i-03a0c086c5df69bcc-2
518 	SRR8265754 	4253 	done 	2020-04-28 19:30:00.419963 	2020-04-28 20:03:00.442220 	i-0eeed28abe4bb5beb-1
517 	SRR8265749 	4254 	done 	2020-04-28 19:30:10.035990 	2020-04-29 02:01:52.180736 	i-0791c8ef7e1075e32-2
517 	SRR8265749 	4255 	aligning 	2020-04-28 19:30:15.513049 	None 	i-08354a520ddfe5521-1
517 	SRR8265749 	4256 	done 	2020-04-28 19:30:16.667407 	2020-04-29 02:08:42.229413 	i-067358b356475cebd-2
517 	SRR8265749 	4257 	aligning 	2020-04-28 19:30:23.414147 	None 	i-0269c458b8602259f-1
517 	SRR8265749 	4258 	aligning 	2020-04-28 19:30:25.699544 	None 	i-0426e7357c0a57951-1
517 	SRR8265749 	4259 	done 	2020-04-28 19:30:46.867655 	2020-04-29 02:07:42.708255 	i-0c05e3eec72c5db4e-1
517 	SRR8265749 	4260 	aligning 	2020-04-28 19:30:47.427788 	None 	i-03a0c086c5df69bcc-1
517 	SRR8265749 	4261 	aligning 	2020-04-28 19:30:55.932791 	None 	i-0d465542cf21f7af6-1
517 	SRR8265749 	4262 	done 	2020-04-28 19:31:16.804273 	2020-04-29 02:09:07.508066 	i-06398526b82f8e278-2
517 	SRR8265749 	4263 	done 	2020-04-28 19:36:14.849212 	2020-04-29 01:46:00.180307 	i-008869062ee3a9250-1


521 	SRR8265752 	4321 	aligning 	2020-04-28 19:47:21.026223 	None 	i-09e241e5fa6aabbe2-1
521 	SRR8265752 	4322 	aligning 	2020-04-28 19:47:31.367470 	None 	i-0d7ef28866fa7cb18-2
521 	SRR8265752 	4323 	aligning 	2020-04-28 19:47:48.627596 	None 	i-0d55c784bfa06f4f4-1
521 	SRR8265752 	4324 	aligning 	2020-04-28 19:47:50.363579 	None 	i-0d6e677a084bba0e1-2
521 	SRR8265752 	4325 	aligning 	2020-04-28 19:48:18.516759 	None 	i-03d18f586c9847a77-2
521 	SRR8265752 	4326 	aligning 	2020-04-28 19:48:41.104085 	None 	i-08e3b76e648de0fa0-2
521 	SRR8265752 	4327 	aligning 	2020-04-28 19:48:55.510375 	None 	i-0ae6b23d890fd8d1d-1
521 	SRR8265752 	4328 	aligning 	2020-04-28 19:49:13.248172 	None 	i-0029ea562f2d47eeb-1
520 	SRR8265751 	4329 	aligning 	2020-04-28 19:49:23.208134 	None 	i-005384031db4e9f6e-2
520 	SRR8265751 	4330 	aligning 	2020-04-28 19:49:27.191770 	None 	i-0e9bb58ae5ca6347c-1
520 	SRR8265751 	4331 	aligning 	2020-04-28 19:49:56.551597 	None 	i-07bd4145fc0577bdf-1
520 	SRR8265751 	4332 	aligning 	2020-04-28 19:49:56.902769 	None 	i-01cc9b764f9dd34a2-2
520 	SRR8265751 	4333 	aligning 	2020-04-28 19:49:59.810305 	None 	i-0db0395459f485686-1
520 	SRR8265751 	4334 	aligning 	2020-04-28 19:50:12.507097 	None 	i-0ce906d01776d145d-2
520 	SRR8265751 	4335 	aligning 	2020-04-28 19:50:36.268972 	None 	i-00330b7680aba8ddf-2
520 	SRR8265751 	4336 	aligning 	2020-04-28 19:50:38.864912 	None 	i-02b94413b29366df4-2

523 	SRR8265746 	4425 	aligning 	2020-04-28 20:02:55.783361 	None 	i-02347284084da0393-1
523 	SRR8265746 	4426 	aligning 	2020-04-28 20:03:02.525794 	None 	i-08d4b3013f856be1c-1
523 	SRR8265746 	4427 	aligning 	2020-04-28 20:03:03.828929 	None 	i-0eeed28abe4bb5beb-1
523 	SRR8265746 	4428 	aligning 	2020-04-28 20:03:16.615843 	None 	i-04a7396a74e345219-1
523 	SRR8265746 	4429 	aligning 	2020-04-28 20:03:17.581272 	None 	i-08d4b3013f856be1c-2
523 	SRR8265746 	4430 	aligning 	2020-04-28 20:03:20.369445 	None 	i-098c2258a5d6449c5-1
523 	SRR8265746 	4431 	aligning 	2020-04-28 20:03:25.219570 	None 	i-0de155fa0ebdf1e48-2
523 	SRR8265746 	4432 	aligning 	2020-04-28 20:03:30.143558 	None 	i-07659f3132551daab-2
```


### Performance

Overall, while there were a few bugs mainly related to disk usage on merge nodes, performance of the cluster maintained a high level of usage, this is working quite efficiently.

![Total Run Usage](200427_ab/total_run_usage.png)


## Output
